In [ ]:
#%pip install numpy pandas matplotlib scikit-learn tensorflow seaborn


In [ ]:
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input, LSTM, RepeatVector, TimeDistributed, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
#df1 = pd.read_csv("../../data/dev101_prepared.csv")
#df2 = pd.read_csv("../../data/dev102_prepared.csv")

#df_all = pd.concat([df1, df2], ignore_index=True)
df_all = pd.read_csv("../../data/devAll2_prepared.csv")

# แปลง timestamp และเรียงลำดับ
df_all['timestamp'] = pd.to_datetime(df_all['timestamp'])
df_all = df_all.sort_values(by='timestamp').reset_index(drop=True)
df_all

In [ ]:
print(df_all['devID'].value_counts())

In [ ]:
# Feature ที่จะใช้
numerical_cols = ['soil', 'rain', 'temp', 'humi', 'geo', 'lat', 'lng']

# สร้าง Scaler และฟิตข้อมูล
scaler = MinMaxScaler()
scaled_values = scaler.fit_transform(df_all[numerical_cols])

# เพิ่มคอลัมน์ scaled กลับเข้า DataFrame
for i, col in enumerate(numerical_cols):
    df_all[f'scaled_{col}'] = scaled_values[:, i]

# Save scaler (สำหรับใช้งานจริงในอนาคต)
joblib.dump(scaler, 'scaler2.save')


In [ ]:
SEQUENCE_LENGTH = 30

def create_sequences_by_device(df, sequence_length, numerical_cols):
    sequences = []
    device_ids = []
    timestamps = []

    for dev_id, group in df.groupby('devID'):
        group = group.sort_values('timestamp')
        values = group[[f'scaled_{col}' for col in numerical_cols]].values
        time_vals = group['timestamp'].values

        for i in range(len(values) - sequence_length + 1):
            sequences.append(values[i:i + sequence_length])
            device_ids.append(dev_id)
            timestamps.append(time_vals[i + sequence_length - 1])  

    return np.array(sequences), device_ids, timestamps

X_seq, device_ids, seq_timestamps = create_sequences_by_device(df_all, SEQUENCE_LENGTH, numerical_cols)


In [ ]:
n = len(X_seq)
train_size = int(n * 0.7)
val_size = int(n * 0.2)
test_size = n - train_size - val_size

X_train = X_seq[:train_size]
X_val = X_seq[train_size:train_size + val_size]
X_test = X_seq[train_size + val_size:]

print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

In [ ]:
n_features = len(numerical_cols)
input_shape = (SEQUENCE_LENGTH, n_features)

encoder_inputs = Input(shape=input_shape)
encoded = LSTM(32, activation='relu', return_sequences=False)(encoder_inputs)
decoded = RepeatVector(SEQUENCE_LENGTH)(encoded)
decoded = LSTM(32, activation='relu', return_sequences=True)(decoded)
decoded = TimeDistributed(Dense(n_features))(decoded)

model = Model(encoder_inputs, decoded)
model.compile(optimizer=Adam(0.001), loss='mse')

model.summary()

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    X_train, X_train,
    validation_data=(X_val, X_val),
    epochs=5, #90
    batch_size=32,
    callbacks=[early_stopping],
    verbose=1
)

# Save model
model.save('lstm_autoencoder_model2.h5')

# Plot loss
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.title('Loss During Training')
plt.grid(True)
plt.show()


In [ ]:
# Predict test set
X_test_pred = model.predict(X_test)
test_reconstruction_error = np.mean(np.mean(np.square(X_test - X_test_pred), axis=1), axis=1)

# แสดงค่า loss บน test set
test_loss = np.mean(test_reconstruction_error)
print(f"Test Reconstruction Loss: {test_loss}")


# Test

In [ ]:
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import load_model
from sklearn.decomposition import PCA

df_test = pd.read_csv('../../data/devAll2_prepared.csv')
df_original_features = df_test.copy() 

df_test['timestamp'] = pd.to_datetime(df_test['timestamp'])
df_test = df_test.sort_values(by='timestamp').reset_index(drop=True)

scaler = joblib.load('scaler2.save')
numerical_cols = ['soil', 'rain', 'temp', 'humi', 'geo', 'lat', 'lng']

missing_cols = [col for col in numerical_cols if col not in df_test.columns]
if missing_cols:
    raise ValueError(f"Missing columns in input CSV: {missing_cols}")

scaled_values = scaler.transform(df_test[numerical_cols])
for i, col in enumerate(numerical_cols):
    df_test[f'scaled_{col}'] = scaled_values[:, i]

# === sequence for predict ===
SEQUENCE_LENGTH = 30

def create_sequences(df, sequence_length, numerical_cols):
    sequences, device_ids, timestamps = [], [], []
    # เก็บ index เริ่มต้นของแต่ละ sequence เพื่อเชื่อมโยงกับ df_test
    start_indices = []

    for dev_id, group in df.groupby('devID'):
        group = group.sort_values('timestamp')
        values = group[[f'scaled_{col}' for col in numerical_cols]].values
        time_vals = group['timestamp'].values
        
        # เพิ่มคอลัมน์ original_index เพื่อให้สามารถเชื่อมโยงกลับไปยัง df_test ได้
        group = group.reset_index(drop=False).rename(columns={'index': 'original_df_index'})

        for i in range(len(values) - sequence_length + 1):
            sequences.append(values[i:i + sequence_length])
            device_ids.append(dev_id)
            timestamps.append(time_vals[i + sequence_length - 1]) # timestamp ของจุดสุดท้ายใน sequence
            start_indices.append(group['original_df_index'].iloc[i + sequence_length - 1]) # เก็บ original index ของจุดสุดท้าย

    return np.array(sequences), device_ids, timestamps, start_indices

# สร้าง sequence
X_seq_test, device_ids_test, timestamps_test, start_indices_test = create_sequences(df_test, SEQUENCE_LENGTH, numerical_cols)

# === 5. โหลดโมเดลและ predict ===
model = load_model('lstm_autoencoder_model2.h5', compile=False)
X_pred = model.predict(X_seq_test)

# === 6. คำนวณ reconstruction error ===
reconstruction_error = np.mean(np.mean(np.square(X_seq_test - X_pred), axis=1), axis=1)

# === 7. สร้าง DataFrame เก็บผล ===
df_results = pd.DataFrame({
    'devID': device_ids_test,
    'timestamp': timestamps_test,
    'reconstruction_error': reconstruction_error,
    'original_df_index': start_indices_test # เพิ่มคอลัมน์นี้เพื่อ merge กับ df_original_features
})

# === 8. คำนวณ thresholds และกำหนด Anomaly Level ===
threshold_warning = 0.01
threshold_critical = 0.0225

def classify_anomaly(err):
    if err > threshold_critical:
        return 'critical'
    elif err > threshold_warning:
        return 'warning'
    else:
        return 'normal'

df_results['anomaly_level'] = df_results['reconstruction_error'].apply(classify_anomaly)

print(f"95th percentile (Warning) threshold = {threshold_warning:.5f}")
print(f"99th percentile (Critical) threshold = {threshold_critical:.5f}")
print("\n--- Anomaly Level Counts ---")
print(df_results['anomaly_level'].value_counts())

In [ ]:
# --- การรวมข้อมูลเพื่อการวิเคราะห์เพิ่มเติม ---
# เราจะ merge df_results กับ df_original_features โดยใช้ 'original_df_index'
# เพื่อให้มีคอลัมน์ features เดิมที่ไม่ถูก scale อยู่ใน DataFrame เดียวกัน
# สร้างคอลัมน์ 'index' ใน df_original_features ให้ตรงกับ 'original_df_index'
# Remove duplicate columns in df_original_features (keep only one 'original_df_index')
df_original_features = df_original_features.loc[:, ~df_original_features.columns.duplicated()]

# Ensure 'original_df_index' exists and is unique
if 'original_df_index' not in df_original_features.columns:
    df_original_features = df_original_features.reset_index().rename(columns={'index': 'original_df_index'})

# Convert timestamp to datetime64[ns] for merge compatibility
df_original_features['timestamp'] = pd.to_datetime(df_original_features['timestamp'])

# Merge ข้อมูล
df_analysis = pd.merge(df_results, df_original_features, on=['devID', 'timestamp', 'original_df_index'], how='left')

# ตรวจสอบว่ามีค่า NaN หลังจาก Merge หรือไม่ (ถ้ามี แสดงว่า timestamp/devID ไม่ตรงกันพอดี)
if df_analysis.isnull().any().any():
    print("\nWarning: Missing values found in merged DataFrame. Ensure timestamps and devIDs match for accurate feature analysis.")

# --- เริ่มการสร้าง Visualization ---
# ตั้งค่า palette สำหรับ Anomaly Levels
palette_anomaly = {'normal': 'green', 'warning': 'orange', 'critical': 'red'}

# --- 1. Distribution of Reconstruction Error ---
plt.figure(figsize=(10, 6))
sns.histplot(df_results['reconstruction_error'], bins=200, kde=True, color='purple', label='Reconstruction Error')
plt.axvline(threshold_warning, color='orange', linestyle='--', label=f'Warning Threshold ({threshold_warning:.5f})')
plt.axvline(threshold_critical, color='red', linestyle='--', label=f'Critical Threshold ({threshold_critical:.5f})')
plt.title('Distribution of Reconstruction Error (Anomaly Score from Autoencoder)')
plt.xlabel('Reconstruction Error')
plt.ylabel('Frequency')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# --- 2. Anomaly Level Counts ---
plt.figure(figsize=(7, 5))
sns.countplot(data=df_results, x='anomaly_level', order=['normal', 'warning', 'critical'], palette=palette_anomaly)
plt.title('Amount of Data in Each Anomaly Level')
plt.xlabel('Anomaly Level')
plt.ylabel('Count')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# --- 3. Reconstruction Error Over Time with Anomaly Levels (All Devices) ---
plt.figure(figsize=(15, 7))
sns.scatterplot(
    data=df_results, x='timestamp', y='reconstruction_error',
    hue='anomaly_level', palette=palette_anomaly, alpha=0.7, s=15,
    hue_order=['normal', 'warning', 'critical']
)
plt.axhline(threshold_warning, color='orange', linestyle='--', label=f'Warning Threshold ({threshold_warning:.5f})')
plt.axhline(threshold_critical, color='red', linestyle='--', label=f'Critical Threshold ({threshold_critical:.5f})')
plt.title('Reconstruction Error over Time with Anomaly Levels (All Device IDs)')
plt.xlabel('Time')
plt.ylabel('Reconstruction Error')
plt.xticks(rotation=45)
plt.legend(title='Anomaly Level')
plt.tight_layout()
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(15, 7))
sns.scatterplot(
    data=df_results[df_results['devID'] == 101.0], 
    x='timestamp', y='reconstruction_error',
    hue='anomaly_level', palette=palette_anomaly, alpha=0.7, s=15,
    hue_order=['normal', 'warning', 'critical']
)
plt.axhline(threshold_warning, color='orange', linestyle='--', label=f'Warning Threshold ({threshold_warning:.5f})')
plt.axhline(threshold_critical, color='red', linestyle='--', label=f'Critical Threshold ({threshold_critical:.5f})')
plt.title('Reconstruction Error over Time with Anomaly Levels (Device 101)')
plt.xlabel('Time')
plt.ylabel('Reconstruction Error')
plt.xticks(rotation=45)
plt.legend(title='Anomaly Level')
plt.tight_layout()
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(15, 7))
sns.scatterplot(
    data=df_results[df_results['devID'] == 102.0], 
    x='timestamp', y='reconstruction_error',
    hue='anomaly_level', palette=palette_anomaly, alpha=0.7, s=15,
    hue_order=['normal', 'warning', 'critical']
)
plt.axhline(threshold_warning, color='orange', linestyle='--', label=f'Warning Threshold ({threshold_warning:.5f})')
plt.axhline(threshold_critical, color='red', linestyle='--', label=f'Critical Threshold ({threshold_critical:.5f})')
plt.title('Reconstruction Error over Time with Anomaly Levels (Device 101)')
plt.xlabel('Time')
plt.ylabel('Reconstruction Error')
plt.xticks(rotation=45)
plt.legend(title='Anomaly Level')
plt.tight_layout()
plt.grid(True)
plt.show()


In [ ]:
# --- 4. Box Plot of Original Features by Anomaly Level ---
print("\n--- Box Plots of Original Features by Anomaly Level ---")
# Optional: define boxprops if you want to customize box appearance
# boxprops = dict(edgecolor='black', linewidth=1.5)

for feature in numerical_cols: # ใช้ numerical_cols เพื่อเข้าถึงคอลัมน์ที่ไม่ถูก scale
    plt.figure(figsize=(9, 6))
    sns.boxplot(
        data=df_analysis,
        x='anomaly_level',
        y=feature,
        order=['normal', 'warning', 'critical'],
        palette=palette_anomaly
    )
    plt.title(f'Boxplot of {feature} by Anomaly Level', fontsize=14)
    plt.xlabel('Anomaly Level', fontsize=12)
    plt.ylabel(feature, fontsize=12)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

print("\nAll Box Plots for Original Features successfully created!")

In [ ]:
# --- 5.1. Bar Plot: จำนวนข้อมูลแต่ละ devID ---
plt.figure(figsize=(8, 4))
df_results['devID'].value_counts().sort_index().plot(kind='bar', color='skyblue')
plt.title('Amount of devID in Sequence')
plt.xlabel('devID')
plt.ylabel('จำนวน Sequence')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# --- 5.2. Dynamic Thresholds (Percentile) ---
threshold_warning = np.percentile(df_results['reconstruction_error'], 95)
threshold_critical = np.percentile(df_results['reconstruction_error'], 99)
print(f"Dynamic 95th percentile (Warning) threshold = {threshold_warning:.5f}")
print(f"Dynamic 99th percentile (Critical) threshold = {threshold_critical:.5f}")

# --- 5.3. PCA Plot of Last Sequence Features by Anomaly Level ---
# X_seq_test มี shape (num_sequences, SEQUENCE_LENGTH, num_features)
# เราจะใช้ค่าจาก timestamp สุดท้าย (index SEQUENCE_LENGTH - 1)
X_features_last_step = X_seq_test[:, -1, :] # นี่คือ features ที่ scaled แล้วของ timestamp สุดท้ายในแต่ละ sequence

pca_autoencoder = PCA(n_components=2)
X_pca_autoencoder = pca_autoencoder.fit_transform(X_features_last_step)

plt.figure(figsize=(10, 7))
sns.scatterplot(x=X_pca_autoencoder[:, 0], y=X_pca_autoencoder[:, 1],
                hue=df_results['anomaly_level'], style=df_results['devID'],
                palette=palette_anomaly, alpha=0.7, s=30,
                hue_order=['normal', 'warning', 'critical'])
plt.title('PCA Plot of Last Sequence Features\nColored by Anomaly Level & Device ID (Autoencoder)')
plt.xlabel(f'Principal Component 1 ({pca_autoencoder.explained_variance_ratio_[0]*100:.2f}%)')
plt.ylabel(f'Principal Component 2 ({pca_autoencoder.explained_variance_ratio_[1]*100:.2f}%)')
plt.legend(title='Anomaly Level / devID', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()

print("\nBar Plot of devID count, dynamic thresholds, and PCA Plot successfully created!")

# Sumulate in Device

In [ ]:
import pandas as pd
import numpy as np
import joblib
from tensorflow.keras.models import load_model
import collections # สำหรับ deque
import matplotlib.pyplot as plt
import seaborn as sns

scaler = joblib.load('scaler2.save')
model = load_model('lstm_autoencoder_model2.h5', compile=False)

SEQUENCE_LENGTH = 30
numerical_cols = ['soil', 'rain', 'temp', 'humi', 'geo', 'lat', 'lng']
LAST_N_ROWS = 1000

df_full_data_source = pd.read_csv('../../data/devAll2_prepared.csv')
df_full_data_source['timestamp'] = pd.to_datetime(df_full_data_source['timestamp'])

df_full_data_source = df_full_data_source.sort_values(by=['timestamp']).reset_index(drop=True)

df_stream_data_source = df_full_data_source.tail(LAST_N_ROWS).reset_index(drop=True)

# --- hresholds ---
threshold_warning = 0.01
threshold_critical = 0.02

def classify_anomaly(err):
    if err > threshold_critical:
        return 'critical'
    elif err > threshold_warning:
        return 'warning'
    else:
        return 'normal'

# --- การเตรียมบัฟเฟอร์สำหรับแต่ละ devID ---
device_buffers = {} 
device_results = collections.defaultdict(list) 

print(f"--- Starting Real-time Anomaly Detection Simulation (Last {LAST_N_ROWS} Rows by Timestamp) ---")
print(f"Warning Threshold: {threshold_warning:.5f}")
print(f"Critical Threshold: {threshold_critical:.5f}")
print("-" * 50)

# ลูปผ่านข้อมูลเพื่อจำลองการรับค่าทีละแถว
current_dev_id_processing = None

for index, row in df_stream_data_source.iterrows():
    dev_id = row['devID']
    current_timestamp = row['timestamp']
    current_features = row[numerical_cols].values 
    
    if dev_id not in device_buffers:
        device_buffers[dev_id] = collections.deque(maxlen=SEQUENCE_LENGTH)

    scaled_features = scaler.transform(current_features.reshape(1, -1))[0]
    
    device_buffers[dev_id].append(scaled_features)
    
    if len(device_buffers[dev_id]) == SEQUENCE_LENGTH:
        input_sequence = np.array(device_buffers[dev_id]).reshape(1, SEQUENCE_LENGTH, len(numerical_cols))

        predicted_sequence = model.predict(input_sequence, verbose=0)

        error = np.mean(np.mean(np.square(input_sequence - predicted_sequence), axis=1), axis=1)[0]

        anomaly_level = classify_anomaly(error)

        print(f"  DevID: {dev_id}, Time: {current_timestamp}, Error: {error:.6f}, Level: {anomaly_level.upper()}")

        device_results['devID'].append(dev_id)
        device_results['timestamp'].append(current_timestamp)
        device_results['reconstruction_error'].append(error)
        device_results['anomaly_level'].append(anomaly_level)

        for col in numerical_cols:
            if col not in device_results:
                device_results[col] = []
            device_results[col].append(row[col])

print("\n--- Real-time Anomaly Detection Simulation Complete ---")

df_realtime_results = pd.DataFrame(device_results)

# --- การพลอตผลลัพธ์ (ถ้ามีข้อมูล) ---
if not df_realtime_results.empty:
    print("\n--- Summary of Real-time Anomaly Detection Results ---")
    print(df_realtime_results['anomaly_level'].value_counts())

    palette_anomaly = {'normal': 'green', 'warning': 'orange', 'critical': 'red'}

    # 1. Distribution of Reconstruction Error (Real-time)
    plt.figure(figsize=(10, 6))
    sns.histplot(df_realtime_results['reconstruction_error'], bins=50, kde=True, color='purple', label='Reconstruction Error')
    plt.axvline(threshold_warning, color='orange', linestyle='--', label=f'Warning Threshold ({threshold_warning:.5f})')
    plt.axvline(threshold_critical, color='red', linestyle='--', label=f'Critical Threshold ({threshold_critical:.5f})')
    plt.title('Distribution of Reconstruction Error (Real-time Simulation - Last 1000 Rows)')
    plt.xlabel('Reconstruction Error')
    plt.ylabel('Frequency')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # 2. Anomaly Level Counts (Real-time)
    plt.figure(figsize=(7, 5))
    sns.countplot(data=df_realtime_results, x='anomaly_level', order=['normal', 'warning', 'critical'], palette=palette_anomaly)
    plt.title('Amount of Data in Each Anomaly Level (Real-time Simulation - Last 1000 Rows)')
    plt.xlabel('Anomaly Level')
    plt.ylabel('Count')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

    # 3. Reconstruction Error Over Time with Anomaly Levels (Real-time)
    plt.figure(figsize=(15, 7))
    sns.scatterplot(
        data=df_realtime_results, x='timestamp', y='reconstruction_error',
        hue='devID', # เปลี่ยนเป็น hue='devID' เพื่อดูแยกแต่ละอุปกรณ์
        palette='tab10', # ใช้ palette ที่มีสีหลากหลายสำหรับ devID
        alpha=0.7, s=15,
        style='anomaly_level', # เพิ่ม style เพื่อแยกระดับ Anomaly
        markers={'normal': 'o', 'warning': '^', 'critical': 'X'}, # กำหนดรูปร่าง marker
        hue_order=sorted(df_realtime_results['devID'].unique()) # เรียง devID ใน legend
    )
    plt.axhline(threshold_warning, color='orange', linestyle='--', label=f'Warning Threshold ({threshold_warning:.5f})')
    plt.axhline(threshold_critical, color='red', linestyle='--', label=f'Critical Threshold ({threshold_critical:.5f})')
    plt.title('Reconstruction Error over Time by Device ID (Real-time Simulation - Last 1000 Rows)')
    plt.xlabel('Time')
    plt.ylabel('Reconstruction Error')
    plt.xticks(rotation=45)
    plt.legend(title='Device ID / Anomaly Level', bbox_to_anchor=(1.05, 1), loc='upper left') # ขยับ legend
    plt.tight_layout()
    plt.grid(True)
    plt.show()

    # 4. Box Plot of Original Features by Anomaly Level (Real-time)
    print("\n--- Box Plots of Original Features by Anomaly Level (Real-time Simulation - Last 1000 Rows) ---")
    for feature in numerical_cols:
        plt.figure(figsize=(9, 6))
        sns.boxplot(data=df_realtime_results, x='anomaly_level', y=feature,
                    order=['normal', 'warning', 'critical'], palette=palette_anomaly)
        plt.title(f'Boxplot of {feature} by Anomaly Level (Real-time Simulation - Last 1000 Rows)', fontsize=14)
        plt.xlabel('Anomaly Level', fontsize=12)
        plt.ylabel(feature, fontsize=12)
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.show()
    print("\nAll Box Plots for Original Features successfully created!")

else:
    print("\nNo anomaly detection results were generated for the last 1000 rows. Check data availability and SEQUENCE_LENGTH.")